In [6]:
# Import Block
import cv2
import numpy as np
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import pdist, squareform
from scipy.spatial import distance
from sklearn.cluster import DBSCAN
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import numpy as np
import os


img =cv2.imread('braille_page_2.jpg')

# constants/changables


In [7]:
# Nepali Braille 6-dot encoding map
# Dot numbering:
# 1 4
# 2 5
# 3 6

def dots_to_bit(dots):
    """Convert list of active dots (1-6) to a 6-bit integer."""
    val = 0
    for d in dots:
        val |= 1 << (6 - d)
    return val

braille_map = {
    dots_to_bit([1]): 'अ',
    dots_to_bit([3, 4, 5]): 'आ',
    dots_to_bit([2, 4]): 'इ',
    dots_to_bit([3, 5]): 'ई',
    dots_to_bit([1, 3, 6]): 'उ',
    dots_to_bit([1, 2, 5, 6]): 'ऊ',
    dots_to_bit([1, 5, 6]): 'ऋ',
    dots_to_bit([1, 5]): 'ए',
    dots_to_bit([3, 4]): 'ऐ',
    dots_to_bit([1, 3, 5]): 'ओ',
    dots_to_bit([2, 4, 6]): 'औ',
    dots_to_bit([1, 6]): 'अं',
    dots_to_bit([6]): 'अः',

    dots_to_bit([1, 3]): 'क',
    dots_to_bit([4, 6]): 'ख',
    dots_to_bit([1, 2, 4, 5]): 'ग',
    dots_to_bit([1, 2, 6]): 'घ',
    dots_to_bit([3, 4, 6]): 'ङ',
    dots_to_bit([1, 4]): 'च',
    dots_to_bit([1, 6]): 'छ',
    dots_to_bit([2, 4, 5]): 'ज',
    dots_to_bit([3, 5, 6]): 'झ',
    dots_to_bit([2, 5]): 'ञ',
    dots_to_bit([2, 3, 4, 6]): 'ट',
    dots_to_bit([2, 4, 5, 6]): 'ठ',
    dots_to_bit([1, 2, 4, 6]): 'ड',
    dots_to_bit([1, 2, 3, 4, 6]): 'ढ',
    dots_to_bit([3, 4, 5, 6]): 'ण',
    dots_to_bit([2, 3, 4, 5]): 'त',
    dots_to_bit([1, 4, 5, 6]): 'थ',
    dots_to_bit([1, 4, 5]): 'द',
    dots_to_bit([2, 3, 5, 6]): 'ध',
    dots_to_bit([1, 3, 4, 5]): 'न',
    dots_to_bit([1, 2, 3, 4]): 'प',
    dots_to_bit([2, 3, 5]): 'फ',
    dots_to_bit([1, 2]): 'ब',
    dots_to_bit([4, 5]): 'भ',
    dots_to_bit([1, 3, 4]): 'म',
    dots_to_bit([1, 3, 4, 5, 6]): 'य',
    dots_to_bit([1, 2, 3, 5]): 'र',
    dots_to_bit([1, 2, 3]): 'ल',
    dots_to_bit([1, 2, 3, 6]): 'व',
    dots_to_bit([1, 4, 6]): 'श',
    dots_to_bit([1, 2, 3, 4, 6]): 'ष',
    dots_to_bit([2, 3, 4]): 'स',
    dots_to_bit([1, 2, 5]): 'ह',
    dots_to_bit([1, 2, 3, 4, 5]): 'क्ष',
    dots_to_bit([1, 5, 6]): 'ज्ञ',
}

# Special compound (multi-cell)
braille_map_compound = {
    # 'त्र' uses two cells: [5] + [1,2,4,5,6]
    (dots_to_bit([5]), dots_to_bit([1, 2, 4, 5, 6])): 'त्र',
}

In [8]:
# preprocessing Block
def preprocess_braille_image(
    img_gray,
    method="adaptive",   # "adaptive", "sauvola", or "otsu"
    block_size=31,       # for adaptive/local threshold (odd, e.g. 25–51)
    C=10,                # constant for adaptive/local threshold
    morph_open=3,        # kernel size for opening (3–7)
    morph_close=5,       # kernel size for closing (3–7)
    apply_tophat=False,  # True if embossing subtle
    min_blob_area=5      # small debris threshold
):
    """
    Preprocess Braille grayscale image for dot detection.
    Returns binary image (dots = white, background = black).
    """

    # --- Step 1: Denoise slightly (optional but helps) ---
    img_blur = cv2.GaussianBlur(img_gray, (3, 3), 0)

    # --- Step 2: Binarization ---
    if method == "adaptive":
        bin_img = cv2.adaptiveThreshold(
            img_blur, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV,  # invert → dots white on black
            block_size, C
        )
    elif method == "otsu":
        _, bin_img = cv2.threshold(
            img_blur, 0, 255,
            cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
        )
    elif method == "sauvola":
        # Sauvola via scikit-image if available
        from skimage.filters import threshold_sauvola
        thresh_s = threshold_sauvola(img_blur, window_size=block_size, k=0.2)
        bin_img = (img_blur < thresh_s).astype(np.uint8) * 255
    else:
        raise ValueError("method must be one of: adaptive, otsu, sauvola")

    # --- Step 3: Morphological cleanup ---
    kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (morph_open, morph_open))
    kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (morph_close, morph_close))

    # Opening: remove small noise
    cleaned = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, kernel_open)

    # Closing: fill small holes in dots
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel_close)

    # --- Step 4: Optional top-hat (enhance embossing if faint) ---
    if apply_tophat:
        kernel_th = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        tophat = cv2.morphologyEx(img_gray, cv2.MORPH_TOPHAT, kernel_th)
        cleaned = cv2.bitwise_or(cleaned, (tophat > 0).astype(np.uint8) * 255)

    # --- Step 5: Optional small blob removal ---
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(cleaned, connectivity=8)
    mask = np.zeros_like(cleaned)
    for i in range(1, num_labels):  # skip background
        if stats[i, cv2.CC_STAT_AREA] >= min_blob_area:
            mask[labels == i] = 255

    return mask

def consolidate_braille_dots(binary_img):
    """
    Fix broken or irregular Braille dots before centroid detection.
    Returns cleaned binary image.
    """
    # Step 1. Close small holes inside each dot
    kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    closed = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel_close)

    # Step 2. Remove small specks between dots
    kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel_open)

    # Step 3. Optional: smooth edges slightly
    blurred = cv2.GaussianBlur(opened, (3, 3), 0)
    _, smooth_bin = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY)

    return smooth_bin

def detect_braille_dots(
    binary_img,
    method="connected",      # "connected", "blob", or "log"
    min_area=5,              # adjust based on resolution (~π*r_min²)
    max_area=200,            # reject large merged blobs
    circularity_thresh=0.6,  # for blob detector
    inertia_thresh=0.5,      # for blob detector
    log_min_sigma=1,         # for LoG
    log_max_sigma=4,
    log_num_sigma=10,
    log_threshold=0.05
):
    """
    Detect Braille dot centroids in a preprocessed binary image.
    Returns list of (x, y) coordinates.
    """

    centroids = []

    # --- Option 1: Connected Components ---
    if method == "connected":
        num_labels, labels, stats, ctds = cv2.connectedComponentsWithStats(binary_img, connectivity=8)
        for i in range(1, num_labels):  # skip background
            area = stats[i, cv2.CC_STAT_AREA]
            if min_area <= area <= max_area:
                x, y = ctds[i]
                centroids.append((float(x), float(y)))

    # --- Option 2: OpenCV SimpleBlobDetector ---
    elif method == "blob":
        params = cv2.SimpleBlobDetector_Params()
        params.filterByArea = True
        params.minArea = min_area
        params.maxArea = max_area
        params.filterByCircularity = True
        params.minCircularity = circularity_thresh
        params.filterByInertia = True
        params.minInertiaRatio = inertia_thresh
        params.filterByConvexity = False

        detector = cv2.SimpleBlobDetector_create(params)
        keypoints = detector.detect(binary_img)
        centroids = [(kp.pt[0], kp.pt[1]) for kp in keypoints]

    # --- Option 3: Laplacian of Gaussian (LoG) ---
    elif method == "log":
        from skimage.feature import blob_log
        binary_float = binary_img.astype(np.float32) / 255.0
        blobs = blob_log(binary_float,
                         min_sigma=log_min_sigma,
                         max_sigma=log_max_sigma,
                         num_sigma=log_num_sigma,
                         threshold=log_threshold)
        # Each blob: (y, x, sigma)
        centroids = [(float(x), float(y)) for y, x, s in blobs]

    else:
        raise ValueError("method must be one of: 'connected', 'blob', or 'log'")

    return np.array(centroids, dtype=np.float32)

In [9]:
# Example usage:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
preprocessed = preprocess_braille_image(img_gray, method="sauvola")
# cv2.imshow('Before', img_gray)
# cv2.imshow('Preprocessed', binary)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
cv2.imwrite("braille_page_output_1.jpg",img)
binary = consolidate_braille_dots(preprocessed)
dots = detect_braille_dots(binary, method="connected", min_area=15, max_area=220)
print("Detected dots:", len(dots))
# print(dots)
for (x, y) in dots:
    cv2.circle(img, (int(x), int(y)), 7, (0,0,255), -1)
# Store results
height, width, channels = img.shape
# Create a white image with the same size
output_image = np.ones((height, width, channels), dtype=np.uint8) * 255
for (x, y) in dots:
    cv2.circle(output_image, (int(x), int(y)), 4, (0,0,0), -1)
cv2.imwrite("braille_page_output_2.jpg",binary)
cv2.imwrite("braille_page_output_3.jpg",img)
cv2.imwrite("braille_page_output_4.jpg",output_image)

Detected dots: 48


True

In [ ]:
from ultralytics import YOLO
import cv2
model = YOLO("braille_yolov11_nano.pt")
# model = YOLO("best_2.pt")
# model = YOLO("best_1.pt")
# model = YOLO("best.pt")
# model = YOLO("yolov8n.pt")
# model = YOLO("Braille_Yolov11_old.pt")
# img = cv2.imread("braille_page_2.jpg")
results = model(output_image)
annotated = results[0].plot()

cv2.imshow("Detections", annotated)
cv2.waitKey(0)



0: 192x640 (no detections), 113.2ms
Speed: 11.5ms preprocess, 113.2ms inference, 8.9ms postprocess per image at shape (1, 3, 192, 640)


-1